In [23]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
df=pd.read_csv('./IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df['sentiment']=df['sentiment'].replace({'positive':1,'negative':0})

In [4]:
X=df['review']
y=df['sentiment']

In [5]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [6]:
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def convert_lower(text):
    return text.lower()

def remove_special(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])

def stem_words(text):
    ps = PorterStemmer()
    return ' '.join([ps.stem(word) for word in text.split()])

In [7]:
preprocessing = Pipeline(steps=[
    ('html_clean', FunctionTransformer(lambda x: [clean_html(text) for text in x], validate=False)),
    ('lowercase', FunctionTransformer(lambda x: [convert_lower(text) for text in x], validate=False)),
    ('remove_special_chars', FunctionTransformer(lambda x: [remove_special(text) for text in x], validate=False)),
    ('remove_stopwords', FunctionTransformer(lambda x: [remove_stopwords(text) for text in x], validate=False)),
    ('stemming', FunctionTransformer(lambda x: [stem_words(text) for text in x], validate=False))
])

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_features=3000)

X_train_bow=cv.fit_transform(X_train).toarray()
X_test_bow=cv.fit_transform(X_test).toarray()

pipeline=Pipeline([('preprocessing',preprocessing),
                   ('countVectorization',CountVectorizer(max_features=1000)),
                   ('to_dense', FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
                  ('gaussian',RandomForestClassifier)])

In [26]:
pipeline.fit(X_train,y_train)

AttributeError: 'numpy.ndarray' object has no attribute '_validate_params'

In [ ]:
y_pred=pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(y_test,y_pred))